# Classificando e-mails usando o classificador Naive Bayes

Os classificadores Naive Bayes são, na verdade, um modelo muito popular para filtragem de e-mail. A ingenuidade deles se presta muito bem à análise de dados de texto, onde cada recurso é uma palavra (ou um **saco de
palavras**), e não seria viável modelar a dependência de cada palavra em todas as outras palavra.

In [3]:
import pandas as pd
from sklearn import feature_extraction
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import naive_bayes

In [4]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


Carregar dados a partir do arquivo binário.

In [5]:
#data.to_pickle('data.pkl')  
#df = pd.read_pickle('data.pkl') 
df = pd.read_feather('data.ftr').set_index('path')
df

,text,class
path,,
data/chapter7/beck-s\2001_plan\1,"Guys, attached you will find a final cut on th...",0.0
data/chapter7/beck-s\2001_plan\2,I am still in need of the information regardin...,0.0
data/chapter7/beck-s\2001_plan\3,Attached is a file containing all cost centers...,0.0
data/chapter7/beck-s\2001_plan\4,"Sarah,\n\nBelow is our high level explanation ...",0.0
data/chapter7/beck-s\2001_plan\5,Sally - I will check into the meaning of this ...,0.0
...,...,...
data/chapter7/SH\SA\20030228_spam_2\01396.e80a10644810bc2ae3c1b58c5fd38dfa,"<html>\n\n<head>\n\n<meta http-equiv=""content-...",1.0
data/chapter7/SH\SA\20030228_spam_2\01397.f75f0dd0dd923faefa3e9cc5ecb8c906,This is a multi-part message in MIME format.\n...,1.0
data/chapter7/SH\SA\20030228_spam_2\01398.8ca7045aae4184d56e8509dc5ad6d979,"Dear Subscriber,\n\n\n\nIf I could show you a ...",1.0


## Pré-processando os dados

O Scikit-learn oferece várias opções quando se trata de recursos de codificação de texto. Um dos mais simples métodos de codificação de dados de texto, lembramos, é por **contagem de palavras**: para cada frase, você conta o
número de ocorrências de cada palavra dentro dele. No scikit-learn, isso é feito facilmente usando `CountVectorizer`:

In [6]:
counts = feature_extraction.text.CountVectorizer()
X = counts.fit_transform(df['text'].values)
X.shape

(52076, 643270)

O resultado é uma matriz gigante, que nos diz que coletamos um total de 52.076 e-mails que contêm coletivamente 643.270 palavras diferentes. No entanto, o scikit-learn é inteligente e salvou o dados em uma matriz esparsa:

In [7]:
X

<52076x643270 sparse matrix of type '<class 'numpy.int64'>'
	with 8607632 stored elements in Compressed Sparse Row format>

Para construir o vetor de rótulos de destino (`y`), precisamos acessar os dados no Pandas. Isso pode ser feito tratando o DataFrame como um dicionário, onde os atributo `values` nos dará acesso ao array NumPy subjacente:

In [8]:
y = df['class'].values

## Treinamento no conjunto de dados completo

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

Aqui usamos `MultinomialNB` do módulo `naive_bayes`, que é a versão do
classificador Bayes ingênuo que é mais adequado para lidar com dados categóricos, como contagens de palavras.

In [10]:
model_naive = naive_bayes.MultinomialNB()
model_naive.fit(X_train, y_train)

MultinomialNB()

O classificador é treinado quase instantaneamente e retorna as pontuações tanto do treinamento quanto do Conjunto de teste:

In [11]:
model_naive.score(X_train, y_train)

0.9508641382621219

In [12]:
model_naive.score(X_test, y_test)

0.9442204301075269

E aí está: 94,4% de precisão no conjunto de teste! 

No entanto, e se fôssemos supercríticos com nosso próprio trabalho e quiséssemos melhorar o resultado ainda mais? Há algumas coisas que poderíamos fazer.

## Usando n-grams para melhorar o resultado

Uma coisa a fazer é usar **contagens de $n$-gramas** em vez de contagens de palavras simples. Até agora, confiamos no que é conhecido como **bolsa de palavras**: simplesmente jogamos cada palavra de um e-mail em uma sacola
e contamos o número de suas ocorrências. No entanto, em e-mails reais, a **ordem** em que **palavras aparecem** podem carregar uma grande quantidade de informações!

Isso é exatamente o que as contagens de $n$-gramas estão tentando transmitir. Você pode pensar em um $n$-grama como um frase que tem $n$ palavras. Por exemplo, a frase *A estatística tem seus momentos* contém o
seguintes 1 grama: *Estatísticas*, *tem*, *seu* e *momentos*. Ele também tem os seguintes 2 gramas:
*A estatística tem*, *tem seus* e *seus momentos*. Também tem dois 3 gramas (*Estatística tem seu* e *tem seu momentos*), e apenas um 4-grama.

Podemos dizer ao CountVectorizer para incluir qualquer ordem de $n$-gramas na matriz de recursos por especificando um intervalo para $n$:

In [13]:
counts = feature_extraction.text.CountVectorizer(
    ngram_range=(1, 2)
)
X = counts.fit_transform(df['text'].values)

Em seguida, repetimos todo o procedimento de divisão dos dados e treinamento do classificador:

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [15]:
model_naive = naive_bayes.MultinomialNB()
model_naive.fit(X_train, y_train)

MultinomialNB()

Você deve ter notado que o treinamento está demorando muito mais desta vez. Para nossa alegria, nós descobrir que o desempenho aumentou significativamente:

In [16]:
model_naive.score(X_test, y_test)

0.9706221198156681

No entanto, as contagens de $n$-gramas não são perfeitas. Eles têm a desvantagem de ponderar injustamente documentos mais longos (porque há mais combinações possíveis de formar $n$-gramas).

Para evitar este problema, podemos usar frequências relativas em vez de um simples número de ocorrências. 

## Usando tf-idf para melhorar o resultado

Ela foi chamada de **frequência de termo inverso de documento (tf–idf)**. O que tf–idf faz é basicamente pesar a contagem de palavras por uma medida de quantas vezes elas aparecem em todo o conjunto de dados.

Um efeito colateral útil desse método é a parte idf — a frequência inversa das palavras. este garante que palavras frequentes, como e, o, e mas, tenham  apenas um pequeno peso na classificação.

Aplicamos tf–idf à matriz de recursos chamando fit_transform na matriz `X`:

In [17]:
tfidf = feature_extraction.text.TfidfTransformer()

In [18]:
X_new = tfidf.fit_transform(X)

Não se esqueça de dividir os dados:

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    X_new, y, test_size=0.2, random_state=42
)

Então, quando treinamos e pontuamos o classificador novamente, de repente encontramos uma pontuação notável de 99% de precisão!

In [20]:
model_naive = naive_bayes.MultinomialNB()
model_naive.fit(X_train, y_train)
model_naive.score(X_test, y_test)

0.9908794162826421

Para nos convencer da grandiosidade do classificador, podemos inspecionar a **matriz de confusão**. Isto é uma matriz que mostra, para cada classe, quantas amostras de dados foram classificadas erroneamente como 
pertencente a uma classe diferente.

Os elementos diagonais na matriz nos dizem quantos amostras da classe $i$ foram corretamente classificadas como pertencentes à classe $i$. O fora da diagonal elementos representam classificações erradas:

In [21]:
metrics.confusion_matrix(y_test, model_naive.predict(X_test))

array([[3746,   84],
       [  11, 6575]], dtype=int64)

Isso nos diz que obtivemos 3.746 classificações de classe 0 corretas e 6.575 classificações de classe 1 correto. Confundimos 84 amostras da classe 0 como pertencentes à classe 1 e